In [ ]:
from datetime import datetime
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(project_root)

from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    TrainingArguments,
    Trainer,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    EarlyStoppingCallback,
)
import torch
import mlflow

from train.utils_train import save_model_and_history, plot_history
# from OCR_VQA.data_preparation import VQAProcessor
from custom_dataset.data_preparation import CustomDataProcessor

c:\Users\Stas\anaconda3\envs\ocr_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
experiment_name = 'trocr_train'

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(experiment_name)

2025/04/09 20:41:30 INFO mlflow.tracking.fluent: Experiment with name 'trocr_train' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/778888683745344032', creation_time=1744220490751, experiment_id='778888683745344032', last_update_time=1744220490751, lifecycle_stage='active', name='trocr_train', tags={}>

In [30]:
experiment = mlflow.get_experiment_by_name(experiment_name)
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

In [33]:
runs_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time


# 1. Dataset preparation

In [ ]:
# TrOCRProcessor class wraps image processor class and tokenizer class
dataset_name = 'ocr-dataset'
# dataset_name = os.path.join(project_root, 'custom_dataset', 'data', dataset_name) # For local dataset
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-printed")


data_processor = CustomDataProcessor(processor)
train_dataset, val_dataset, test_dataset, train_size = data_processor(
    dataset_type='S3', # change to 'local' for using local stored dataset
    train_frac=0.95,
    val_frac=0.025,
    dataset_name=dataset_name,
    batch_size=16,
)

# 2. Train

In [ ]:
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-printed")
model = VisionEncoderDecoderModel.from_pretrained("checkpoints/06.04.25_S3_100k_v2/checkpoint-10000")

# Choosing a strategy for text generation
# All strategies can be found here: 
# https://huggingface.co/docs/transformers/v4.48.2/en/main_classes/text_generation#transformers.GenerationConfig

gen_config = dict(
    num_beams=7,
    num_beam_groups=1,
    do_sample=True,
    max_new_tokens=200,
    early_stopping=True, # True stops when num_beams candidates are reached
    temperature=1.5, # T <(>) 1 sharpens (smoothes) probability distribution
    top_k=100, # Only top k candidates with highest probabilities will be considered
    diversity_penalty=0, # The value is substracted from beam score if the token is generated by another group
    repetition_penalty=1.2, # AFAIK Directly multiplied by temperature
    # decoder_start_token_id=processor.tokenizer.bos_token_id,
    # pad_token_id=processor.tokenizer.pad_token_id,
)

model.generation_config.update(**gen_config) # Update existing generation config with new values
model.config.decoder_start_token_id = processor.tokenizer.bos_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [ ]:
# Define parameters

date = datetime.now().strftime("%d.%m.%y")

model_name = f'{date}_{dataset_name}_v1'

output_dir = f'checkpoints/{model_name}'
num_epochs = 10
batch_size = 8
init_learning_rate = 2.0e-6 #1e-5
max_steps = int((train_size / batch_size) * num_epochs)
eval_steps = logging_steps = 1000

# Initialize the optimizer. See this for optimizers:
# https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules
optimizer = torch.optim.AdamW(model.parameters(), lr=init_learning_rate)

# Set up a learning rate scheduler. See this for scheduler types:
# https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/optimizer_schedules#transformers.SchedulerType
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(max_steps * 0), # % of the steps for warmup
    num_training_steps=max_steps,
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

training_hyperparams = TrainingArguments(
    output_dir=output_dir,
    report_to='mlflow',
    run_name=model_name, # for mlflow logging
    # learning_rate=1e-4,
    # lr_scheduler_type='linear', 
    eval_strategy='steps', # evaluate on eval_dataset every eval_steps
    eval_steps=eval_steps,
    eval_accumulation_steps=logging_steps,
    logging_steps=logging_steps, # update steps to perform before output logs
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    # max_steps=max_steps, # Overrides num_train_epochs
    save_total_limit=1, # Save only last checkpoint
    load_best_model_at_end=True, # Save best model
    metric_for_best_model='eval_loss', # Key from dict, returned by compute_metrics, or some predefined values
    greater_is_better=False,
    save_steps=10000,
    # logging_dir='trocr_checkpoints/logs',
    # fp16=True,
    fp16_full_eval=True,
)

In [ ]:
# Training the model

if not os.path.exists(output_dir):
    trainer = Trainer(
        model=model,
        args=training_hyperparams,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        # preprocess_logits_for_metrics=preprocess_logits_for_metrics,
        # compute_metrics=compute_metrics,
        optimizers=(optimizer, lr_scheduler),
        callbacks=[early_stopping]
        # processing_class=processor,
        # data_collator=...
    )
    trainer.train()
else:
    raise ValueError(f"Model '{model_name}' exists, specify another name")

  1%|          | 1000/118560 [09:58<24:48:53,  1.32it/s]

{'loss': 0.3108, 'grad_norm': 2.1685426235198975, 'learning_rate': 2.199613826365429e-06, 'epoch': 0.08}


                                                        
  1%|          | 1000/118560 [12:11<24:48:53,  1.32it/s]

{'eval_loss': 0.24410735070705414, 'eval_runtime': 133.0535, 'eval_samples_per_second': 18.752, 'eval_steps_per_second': 2.345, 'epoch': 0.08}


  2%|▏         | 2000/118560 [21:51<18:28:01,  1.75it/s]  

...

  9%|▉         | 11000/118560 [2:18:00<24:53:03,  1.20it/s]  

{'loss': 0.3071, 'grad_norm': 3.4142446517944336, 'learning_rate': 2.15360017923913e-06, 'epoch': 0.93}


                                                           
  9%|▉         | 11000/118560 [2:20:43<24:53:03,  1.20it/s]

{'eval_loss': 0.2567855417728424, 'eval_runtime': 162.7511, 'eval_samples_per_second': 15.33, 'eval_steps_per_second': 1.917, 'epoch': 0.93}


  9%|▉         | 11113/118560 [2:22:05<19:48:53,  1.51it/s]  

# 3. Save model and history

In [ ]:
with mlflow.start_run(run_name=model_name):

    mlflow.transformers.log_model(trainer.model, artifact_path=model_name)
    mlflow.log_params(training_hyperparams.to_dict())
    metrics = trainer.evaluate()
    mlflow.log_metrics(metrics)

In [ ]:
save_model_and_history(model_name, trainer)

# 4. Evaluation

__1. Plot training history__

In [ ]:
epochs = [i['epoch'] for i in trainer.state.log_history if 'eval_loss' in i]
train_loss = [i['loss'] for i in trainer.state.log_history if 'loss' in i]
val_loss = [i['eval_loss'] for i in trainer.state.log_history if 'eval_loss' in i]
# val_cer = [10 * i['eval_cer'] for i in trainer.state.log_history if 'eval_cer' in i]

hist = {'train_loss': train_loss, 'val_loss': val_loss}#, 'val_cer * 10': val_cer}

In [ ]:
fig = plot_history(
    epochs,
    hist,
    run_name=model_name,
    figsize=(15, 10),
)
